In [3]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from google.cloud import storage
import pandas as pd

In [6]:
def download_csv(csv_name):
    bucket_path = "de2022_f1_jmc"
    client = storage.Client()
    bucket = client.bucket(bucket_path)
    if isinstance(csv_name, list):
        for csv in csv_name:
            bucket.blob(f'data/{csv}').download_to_filename(f"../data/{csv}")
    else:
        bucket.blob(f'data/{csv}').download_to_filename(f"../data/{csv_name}")

# download_csv(["races.csv","drivers.csv", "results.csv", "circuits.csv"])

f1_races_path = "./data/f1_stats/races.csv"
f1_score_path = "./data/f1_stats/results.csv"
f1_divers_path = "./data/f1_stats/drivers.csv"
f1_circuits_path = "./data/f1_stats/circuits.csv"

In [14]:
# Create Spark session
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("app") \
    .getOrCreate()

def get_data(file_name):
    return spark.read.csv(file_name, header='true');

races = get_data(f1_races_path);
scores = get_data(f1_score_path);
drivers = get_data(f1_divers_path);
circuits = get_data(f1_circuits_path);

df = scores.join(races.alias("races"), scores.raceId ==  races.raceId,"left") \
     .join(drivers, scores.driverId == drivers.driverId, "left") \
     .join(races.alias("prevRace"), col("prevRace.raceId") == scores.raceId - 1, "left") \
     .join(circuits, col("races.circuitId") == circuits.circuitId, "left") \
     .filter(col("races.year") == 2021) \
     .filter(scores.position == 1) \
     .select(scores.raceId, circuits.location, col("races.name"), drivers.surname, to_date(col("races.date"),"yyyy-MM-dd").alias("to_date"), to_date(col("prevRace.date"), "yyyy-MM-dd").alias("from_date")) \
     .orderBy(col("to_date"));

df.show()

+------+------------+--------------------+----------+----------+----------+
|raceId|    location|                name|   surname|   to_date| from_date|
+------+------------+--------------------+----------+----------+----------+
|  1052|      Sakhir|  Bahrain Grand Prix|  Hamilton|2021-03-28|      null|
|  1053|       Imola|Emilia Romagna Gr...|Verstappen|2021-04-18|2021-03-28|
|  1054|    Portimão|Portuguese Grand ...|  Hamilton|2021-05-02|2021-04-18|
|  1055|    Montmeló|  Spanish Grand Prix|  Hamilton|2021-05-09|2021-05-02|
|  1056| Monte-Carlo|   Monaco Grand Prix|Verstappen|2021-05-23|2021-05-09|
|  1057|        Baku|Azerbaijan Grand ...|     Pérez|2021-06-06|2021-05-23|
|  1059|Le Castellet|   French Grand Prix|Verstappen|2021-06-20|2021-06-27|
|  1058|   Spielberg|  Styrian Grand Prix|Verstappen|2021-06-27|2021-06-06|
|  1060|   Spielberg| Austrian Grand Prix|Verstappen|2021-07-04|2021-06-20|
|  1061| Silverstone|  British Grand Prix|  Hamilton|2021-07-18|2021-07-04|
|  1062|    

In [16]:
# conf = spark.sparkContext._jsc.hadoopConfiguration()
# conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
# conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# # Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
# bucket = "de_jads_temp-401"
# spark.conf.set('temporaryGcsBucket', bucket)

# df \
#   .write.format('bigquery') \
#   .option('table', 'de2022-362622.assignmentDatasets.racesSchedule') \
#   .mode("overwrite") \
#   .save()

df.write.csv("./data/schedule.csv")